# 🫀 CNN1D Autoencoder para Detección de Anomalías en ECG

Este notebook implementa un **Autoencoder 1D CNN** para detección de anomalías en señales ECG mediante **entrenamiento no supervisado**.

**Características principales:**
- Arquitectura CNN pura: CNN1D encoder + CNN1D decoder (ConvTranspose1d o Upsampling)
- Entrenamiento no supervisado: solo con ejemplos normales
- Detección de anomalías mediante error de reconstrucción
- Selección automática de umbral de anomalía
- Evaluación con métricas de clasificación (precision, recall, f1, etc.)
- Datos preprocesados desde `Datos_no_supervisados/tensors_200hz` (archivos .pt)
- Integración con MLflow para tracking de experimentos
- Orquestación con Prefect 2.x
- Soporte automático para GPU (RTX 5080 compatible)

> ⚠️ **IMPORTANTE EN WINDOWS:** Ejecuta la celda de **Setup DLLs CUDA** (celda 2) **ANTES** de la celda de imports. Esto es necesario para que PyTorch pueda cargar las DLLs de CUDA correctamente.

> ▶️ **Instrucciones:** 
> 1. Ejecuta la celda de **Setup DLLs CUDA** primero
> 2. Configura los parámetros en la sección de **CONFIGURACIÓN GENERAL**
> 3. Ajusta la ruta `DATA_DIR` a tu carpeta de datos (debe apuntar a `Datos_no_supervisados/tensors_200hz`)
> 4. Ejecuta todas las demás celdas en orden


## 📋 Índice

1. **Setup CUDA y dependencias** - Configuración de DLLs y librerías
2. **Configuración general** - Imports, semillas, dispositivo, hiperparámetros
3. **Carga y preparación de datos** - Funciones para cargar desde `tensors_200hz` (filtrar normales para train)
4. **Definición del modelo Autoencoder CNN1D** - Arquitectura encoder-decoder CNN
5. **Funciones de entrenamiento y evaluación** - Loops de entrenamiento y cálculo de error de reconstrucción
6. **Integración con MLflow** - Configuración y logging
7. **Orquestación con Prefect** - Flujo principal con Prefect
8. **Selección de umbral de anomalía** - Calcular umbral óptimo
9. **Evaluación final** - Evaluar en validación y test con umbral
10. **Guardado de modelo** - Exportar modelo y umbral


---

## 1. ⚙️ Setup CUDA y Dependencias


In [1]:
# ========================================
# 🔧 Setup RTX 5080 — dependencias + CUDA DLL
# Ejecuta una sola vez (o tras actualizar drivers/librerías)
# ========================================
import os
import sys
import subprocess
from pathlib import Path
from textwrap import dedent

print(f"Python: {sys.executable}")
print(f"Working dir: {Path.cwd().resolve()}")

# Rutas candidatas para DLLs de CUDA
CUDA_CANDIDATES = [
    os.environ.get("CUDA_PATH"),
    os.environ.get("CUDA_PATH_V12_8"),
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8",
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin",
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\libnvvp",
    r"C:\Program Files\NVIDIA\CUDNN",
]

# Añadir rutas DLL en Windows (necesario antes de importar torch)
added = []
if hasattr(os, "add_dll_directory"):
    for candidate in CUDA_CANDIDATES:
        if not candidate:
            continue
        path = Path(candidate)
        if path.is_dir():
            try:
                os.add_dll_directory(str(path))
                added.append(str(path))
            except (FileNotFoundError, OSError):
                pass

if added:
    print("DLL directories añadidos:")
    for path in added:
        print(f"  - {path}")

# Instalar dependencias base si no están instaladas
BASE_PACKAGES = [
    "mlflow>=2.16",
    "prefect>=3",
    "scikit-learn",
    "matplotlib",
    "pandas",
    "numpy",
    "seaborn",
    "ipywidgets",
]

def pip_install(spec: str) -> None:
    module_name = spec.split("==")[0].split("[")[0].replace("-", "_")
    try:
        __import__(module_name)
        print(f"✔ {spec} ya instalado")
    except Exception:
        print(f"⏳ Instalando {spec} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", spec])

for pkg in BASE_PACKAGES:
    pip_install(pkg)

# Comando para instalar PyTorch nightly con CUDA 12.8 (para RTX 5080)
TORCH_INSTALL_CMD = [
    sys.executable,
    "-m",
    "pip",
    "install",
    "--upgrade",
    "--pre",
    "torch",
    "torchvision",
    "torchaudio",
    "--index-url",
    "https://download.pytorch.org/whl/nightly/cu128",
]

def ensure_torch_cuda() -> "tuple[object | None, dict]":
    """Importa torch, o instala la nightly cu128 si hace falta."""
    info: dict[str, str | float | bool] = {}
    try:
        import torch  # type: ignore
        info["torch_version"] = getattr(torch, "__version__", "desconocida")
        info["cuda_version"] = getattr(getattr(torch, "version", object()), "cuda", "desconocida")
        info["cuda_available"] = bool(torch.cuda.is_available())
        if "cu128" not in info["torch_version"] and not str(info["cuda_version"]).startswith("12.8"):
            raise RuntimeError(
                f"Build {info['torch_version']} no es cu128. Se reinstalará la nightly para RTX 5080."
            )
        return torch, info
    except Exception as err:
        print("⚠️ Torch no usable todavía:", err)
        print("   Desinstalando PyTorch corrupto...")
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision", "torchaudio"])
        print("   Instalando nightly cu128 desde PyTorch (puede tardar).")
        subprocess.check_call(TORCH_INSTALL_CMD)
        print("\n" + "="*60)
        print("⚠️ IMPORTANTE: PyTorch fue reinstalado.")
        print("   DEBES REINICIAR EL KERNEL DE JUPYTER ahora:")
        print("   Kernel → Restart Kernel")
        print("   Luego ejecuta esta celda de nuevo.")
        print("="*60)
        import importlib
        import time
        time.sleep(2)
        importlib.invalidate_caches()
        try:
            import torch  # type: ignore
            info["torch_version"] = getattr(torch, "__version__", "desconocida")
            info["cuda_version"] = getattr(getattr(torch, "version", object()), "cuda", "desconocida")
            info["cuda_available"] = bool(torch.cuda.is_available())
            return torch, info
        except Exception as e2:
            print(f"\n❌ No se pudo importar PyTorch después de reinstalar: {e2}")
            print("   Por favor, REINICIA EL KERNEL y ejecuta esta celda de nuevo.")
            raise RuntimeError("Reinicia el kernel de Jupyter y ejecuta esta celda de nuevo.") from e2

# Intentar importar/instalar PyTorch
torch, torch_info = ensure_torch_cuda()

print("\nTorch info:")
for k, v in torch_info.items():
    print(f"  - {k}: {v}")

if torch_info.get("cuda_available"):
    try:
        gpu_name = torch.cuda.get_device_name(0)
        cc = torch.cuda.get_device_properties(0)
        print(f"GPU detectada: {gpu_name} | SM {cc.major}{cc.minor}")
    except Exception as e:
        print("⚠️ CUDA disponible pero no se pudo consultar GPU:", e)
else:
    print(dedent(
        """
        ⚠️ CUDA sigue inactiva. Revisa drivers / reinicia kernel tras la instalación.
        Si el problema continúa, ejecuta manualmente:
          pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
        """
    ))


Python: c:\Python311\python.exe
Working dir: S:\Proyecto final\Books
DLL directories añadidos:
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\libnvvp
⏳ Instalando mlflow>=2.16 ...
⏳ Instalando prefect>=3 ...
⏳ Instalando scikit-learn ...
✔ matplotlib ya instalado
✔ pandas ya instalado
✔ numpy ya instalado
✔ seaborn ya instalado
✔ ipywidgets ya instalado

Torch info:
  - torch_version: 2.10.0.dev20251124+cu128
  - cuda_version: 12.8
  - cuda_available: True
GPU detectada: NVIDIA GeForce RTX 5080 | SM 120


In [2]:
# ========================================
# Imports y dependencias
# ========================================
# ⚠️ IMPORTANTE: Ejecuta la celda anterior (Setup DLLs) antes de esta celda
# torch ya está importado en la celda anterior
import random
import json
import time
from pathlib import Path
from typing import Tuple, Dict, List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# torch ya está importado en la celda anterior, solo importamos los submódulos
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    accuracy_score,
    classification_report,
    roc_auc_score,
)
import mlflow
import mlflow.pytorch
from prefect import task, flow
from prefect.tasks import NO_CACHE

# Instalar seaborn si no está disponible (para las visualizaciones)
try:
    import seaborn as sns
except ImportError:
    print("⏳ Instalando seaborn...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "seaborn"])
    import seaborn as sns
    print("✓ seaborn instalado")

print("✓ Todos los imports completados")


✓ Todos los imports completados


---

## 2. ⚙️ Configuración General


In [3]:
# ========================================
# CONFIGURACIÓN GENERAL
# ========================================

# --- Rutas y nombres ---
DATA_DIR = Path("../data/Datos_no_supervisados/tensors_200hz")  # TODO: cambiar por la carpeta donde están los datos
EXPERIMENT_NAME = "ecg_cnn_autoencoder"
RUN_NAME = "cnn_ae_ecg_v1"
OUTPUT_DIR = Path("./outputs")                 # Directorio para guardar artefactos

# --- Datos de entrada ---
N_CHANNELS = 3           # cantidad de derivaciones / canales ECG
SEQ_LEN = 2000           # TODO: timesteps por ejemplo (ej: 10 s a 200 Hz => 2000)

# --- Arquitectura CNN Encoder ---
# Cada entrada define una capa Conv1d:
# - enc_out_channels[i]: filtros de la capa i
# - enc_kernel_sizes[i]: tamaño de kernel de la capa i
# - enc_pool_sizes[i]: tamaño del MaxPool1d posterior (si se usa)
enc_out_channels = [16, 32, 64]   # TODO: ajustar filtros por capa
enc_kernel_sizes = [7, 5, 3]      # TODO: ajustar tamaños de kernel
enc_pool_sizes = [2, 2, 2]        # TODO: ajustar factor de pooling por capa (o None si no se usa)

USE_BATCHNORM = True              # aplicar BatchNorm1d después de cada conv
CNN_ACTIVATION = "relu"           # usar ReLU

# --- Arquitectura CNN Decoder ---
# Intentar espejo del encoder (se puede usar ConvTranspose1d o upsampling + Conv1d).
dec_out_channels = [32, 16, N_CHANNELS]   # TODO: espejo del encoder hacia N_CHANNELS
dec_kernel_sizes = [5, 7, 7]             # TODO: tamaños de kernel para decoder
dec_upsample_sizes = [2, 2, 1]           # factores de upsampling (último puede ser 1)

# --- Representación intermedia ---
LATENT_CHANNELS = 64      # canales en la capa más profunda (coincide normalmente con enc_out_channels[-1])

# --- Entrenamiento ---
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
NUM_EPOCHS = 50
WEIGHT_DECAY = 1e-5       # regularización L2 (0.0 si no se quiere)

# --- Umbral de anomalía ---
THRESHOLD_METHOD = "percentile"   # p.ej. "percentile" o "mean_std"
THRESHOLD_PERCENTILE = 95         # si se usa método percentil, ej. 95 o 99

# --- Otros ---
SEED = 42
USE_CUDA = True             # si hay GPU disponible, usarla
ENABLE_CUDNN_BENCHMARK = True

# --- MLflow ---
MLFLOW_TRACKING_URI = None  # None = usa el directorio local (sqlite:///mlflow.db)

# Crear directorio de salida
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Diccionario de configuración (para pasar a funciones)
CONFIG = {
    "DATA_DIR": DATA_DIR,
    "EXPERIMENT_NAME": EXPERIMENT_NAME,
    "RUN_NAME": RUN_NAME,
    "OUTPUT_DIR": OUTPUT_DIR,
    "N_CHANNELS": N_CHANNELS,
    "SEQ_LEN": SEQ_LEN,
    "enc_out_channels": enc_out_channels,
    "enc_kernel_sizes": enc_kernel_sizes,
    "enc_pool_sizes": enc_pool_sizes,
    "USE_BATCHNORM": USE_BATCHNORM,
    "CNN_ACTIVATION": CNN_ACTIVATION,
    "dec_out_channels": dec_out_channels,
    "dec_kernel_sizes": dec_kernel_sizes,
    "dec_upsample_sizes": dec_upsample_sizes,
    "LATENT_CHANNELS": LATENT_CHANNELS,
    "BATCH_SIZE": BATCH_SIZE,
    "LEARNING_RATE": LEARNING_RATE,
    "NUM_EPOCHS": NUM_EPOCHS,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "THRESHOLD_METHOD": THRESHOLD_METHOD,
    "THRESHOLD_PERCENTILE": THRESHOLD_PERCENTILE,
    "SEED": SEED,
    "USE_CUDA": USE_CUDA,
    "ENABLE_CUDNN_BENCHMARK": ENABLE_CUDNN_BENCHMARK,
    "MLFLOW_TRACKING_URI": MLFLOW_TRACKING_URI,
}

print("✓ Configuración cargada:")
print(json.dumps({k: str(v) if isinstance(v, Path) else v for k, v in CONFIG.items()}, indent=2, ensure_ascii=False))


✓ Configuración cargada:
{
  "DATA_DIR": "..\\data\\Datos_no_supervisados\\tensors_200hz",
  "EXPERIMENT_NAME": "ecg_cnn_autoencoder",
  "RUN_NAME": "cnn_ae_ecg_v1",
  "OUTPUT_DIR": "outputs",
  "N_CHANNELS": 3,
  "SEQ_LEN": 2000,
  "enc_out_channels": [
    16,
    32,
    64
  ],
  "enc_kernel_sizes": [
    7,
    5,
    3
  ],
  "enc_pool_sizes": [
    2,
    2,
    2
  ],
  "USE_BATCHNORM": true,
  "CNN_ACTIVATION": "relu",
  "dec_out_channels": [
    32,
    16,
    3
  ],
  "dec_kernel_sizes": [
    5,
    7,
    7
  ],
  "dec_upsample_sizes": [
    2,
    2,
    1
  ],
  "LATENT_CHANNELS": 64,
  "BATCH_SIZE": 64,
  "LEARNING_RATE": 0.001,
  "NUM_EPOCHS": 50,
  "WEIGHT_DECAY": 1e-05,
  "THRESHOLD_METHOD": "percentile",
  "THRESHOLD_PERCENTILE": 95,
  "SEED": 42,
  "USE_CUDA": true,
  "ENABLE_CUDNN_BENCHMARK": true,
  "MLFLOW_TRACKING_URI": null
}


In [4]:
# ========================================
# Configuración de semillas aleatorias y optimizaciones GPU
# ========================================
def set_seed_everywhere(seed: int = 42, enable_cudnn_benchmark: bool = True) -> None:
    """Fija semillas para reproducibilidad y optimiza GPU."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = enable_cudnn_benchmark
        torch.cuda.empty_cache()
        print(f"✓ cuDNN Benchmark: {'Habilitado' if enable_cudnn_benchmark else 'Deshabilitado'}")

set_seed_everywhere(SEED, enable_cudnn_benchmark=CONFIG.get("ENABLE_CUDNN_BENCHMARK", True))
print(f"✓ Semilla fijada: {SEED}")


✓ cuDNN Benchmark: Habilitado
✓ Semilla fijada: 42


In [5]:
# ========================================
# Configuración de dispositivo (GPU/CPU)
# ========================================
def get_device() -> torch.device:
    """Detecta y configura el dispositivo (GPU si está disponible)."""
    if USE_CUDA and torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ GPU detectada: {gpu_name}")
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  PyTorch Version: {torch.__version__}")
    else:
        device = torch.device("cpu")
        print("⚠ GPU no disponible, usando CPU")
    return device

DEVICE = get_device()
print(f"Dispositivo seleccionado: {DEVICE}")


✓ GPU detectada: NVIDIA GeForce RTX 5080
  CUDA Version: 12.8
  PyTorch Version: 2.10.0.dev20251124+cu128
Dispositivo seleccionado: cuda


---

## 3. 📂 Carga y Preparación de Datos


In [6]:
# ========================================
# Función para cargar datos desde tensors_200hz (archivos .pt)
# IMPORTANTE: Para entrenamiento solo usamos ejemplos normales (y == 0)
# ========================================
def load_tensor_data(
    data_dir: Path,
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Carga los datos desde archivos .pt en tensors_200hz.
    
    Para entrenamiento: solo ejemplos normales (y == 0)
    Para validación y test: todos los ejemplos (normales y anómalos)
    
    Args:
        data_dir: Ruta a la carpeta tensors_200hz
        
    Returns:
        Tuple con (X_train_normals, y_train_normals, X_val, y_val, X_test, y_test)
    """
    print("="*70)
    print("📂 CARGANDO DATOS DESDE tensors_200hz")
    print("="*70)
    print(f"Directorio: {data_dir.resolve()}")
    
    # Cargar tensors
    print("\n⏳ Cargando tensors desde disco...")
    X_train = torch.load(data_dir / "X_train.pt", map_location='cpu')
    y_train = torch.load(data_dir / "y_train.pt", map_location='cpu')
    X_val = torch.load(data_dir / "X_val.pt", map_location='cpu')
    y_val = torch.load(data_dir / "y_val.pt", map_location='cpu')
    X_test = torch.load(data_dir / "X_test.pt", map_location='cpu')
    y_test = torch.load(data_dir / "y_test.pt", map_location='cpu')
    
    # Filtrar solo normales para entrenamiento (y == 0)
    print("\n🔍 Filtrando datos de entrenamiento (solo normales)...")
    normal_mask = (y_train == 0)
    X_train_normals = X_train[normal_mask]
    y_train_normals = y_train[normal_mask]
    
    print(f"\n✓ Datos cargados:")
    print(f"  X_train (normales): {X_train_normals.shape} | y_train: {y_train_normals.shape} (todos normales: {(y_train_normals==0).sum().item()})")
    print(f"  X_val:   {X_val.shape} | y_val:   {y_val.shape} (normales: {(y_val==0).sum().item()}, anómalos: {(y_val==1).sum().item()})")
    print(f"  X_test:  {X_test.shape} | y_test:  {y_test.shape} (normales: {(y_test==0).sum().item()}, anómalos: {(y_test==1).sum().item()})")
    
    # Verificar que las features coincidan con CONFIG
    if X_train_normals.shape[-1] != CONFIG["N_CHANNELS"]:
        print(f"⚠️ ADVERTENCIA: X_train tiene {X_train_normals.shape[-1]} canales, pero N_CHANNELS={CONFIG['N_CHANNELS']}")
        print(f"   Ajustando N_CHANNELS a {X_train_normals.shape[-1]}")
        CONFIG["N_CHANNELS"] = X_train_normals.shape[-1]
    
    # Verificar SEQ_LEN
    if len(X_train_normals.shape) >= 2 and X_train_normals.shape[1] != CONFIG["SEQ_LEN"]:
        print(f"⚠️ ADVERTENCIA: X_train tiene {X_train_normals.shape[1]} timesteps, pero SEQ_LEN={CONFIG['SEQ_LEN']}")
        print(f"   Ajustando SEQ_LEN a {X_train_normals.shape[1]}")
        CONFIG["SEQ_LEN"] = X_train_normals.shape[1]
    
    print("="*70)
    
    return X_train_normals, y_train_normals, X_val, y_val, X_test, y_test


def create_dataloaders_from_tensors(
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    X_test: torch.Tensor,
    y_test: torch.Tensor,
    batch_size: int,
    shuffle_train: bool = True,
) -> Tuple[DataLoader, DataLoader, DataLoader]:
    """
    Crea DataLoaders desde tensors.
    
    Returns:
        Tuple con (train_loader, val_loader, test_loader)
    """
    # Crear datasets
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    
    # Crear dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=shuffle_train,
        num_workers=0,  # 0 para Windows
        pin_memory=torch.cuda.is_available(),
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available(),
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available(),
    )
    
    print(f"\n✓ DataLoaders creados:")
    print(f"  Train: {len(train_loader)} batches ({len(train_dataset)} muestras)")
    print(f"  Val:   {len(val_loader)} batches ({len(val_dataset)} muestras)")
    print(f"  Test:  {len(test_loader)} batches ({len(test_dataset)} muestras)")
    
    return train_loader, val_loader, test_loader


---

## 4. 🧠 Definición del Modelo Autoencoder CNN1D


In [7]:
# ========================================
# Clase Autoencoder CNN1D
# ========================================
class CNN1D_Autoencoder(nn.Module):
    """
    Autoencoder 1D CNN para reconstrucción de series temporales ECG.
    
    Arquitectura:
    - Encoder: CNN1D con pooling
    - Decoder: CNN1D con upsampling (ConvTranspose1d o Upsample + Conv1d)
    """
    
    def __init__(
        self,
        n_channels: int,
        seq_len: int,
        enc_out_channels: List[int],
        enc_kernel_sizes: List[int],
        enc_pool_sizes: List[int],
        dec_out_channels: List[int],
        dec_kernel_sizes: List[int],
        dec_upsample_sizes: List[int],
        use_batchnorm: bool = True,
        cnn_activation: str = "relu",
        latent_channels: int = 64,
    ):
        super(CNN1D_Autoencoder, self).__init__()
        
        self.n_channels = n_channels
        self.seq_len = seq_len
        self.enc_out_channels = enc_out_channels
        self.dec_out_channels = dec_out_channels
        self.num_enc_layers = len(enc_out_channels)
        self.num_dec_layers = len(dec_out_channels)
        self.latent_channels = latent_channels
        
        # ========================================
        # Encoder: CNN1D
        # ========================================
        encoder_layers = []
        in_channels = n_channels
        
        for i in range(self.num_enc_layers):
            # Conv1d
            encoder_layers.append(
                nn.Conv1d(
                    in_channels=in_channels,
                    out_channels=enc_out_channels[i],
                    kernel_size=enc_kernel_sizes[i],
                    padding=(enc_kernel_sizes[i] - 1) // 2,  # padding 'same'
                )
            )
            
            # BatchNorm (si está habilitado)
            if use_batchnorm:
                encoder_layers.append(nn.BatchNorm1d(enc_out_channels[i]))
            
            # Activación
            if cnn_activation.lower() == "relu":
                encoder_layers.append(nn.ReLU())
            elif cnn_activation.lower() == "leakyrelu":
                encoder_layers.append(nn.LeakyReLU(0.1))
            else:
                raise ValueError(f"Activación {cnn_activation} no soportada")
            
            # MaxPool (si está definido)
            if enc_pool_sizes[i] is not None and enc_pool_sizes[i] > 1:
                encoder_layers.append(nn.MaxPool1d(kernel_size=enc_pool_sizes[i]))
            
            in_channels = enc_out_channels[i]
        
        self.encoder = nn.Sequential(*encoder_layers)
        
        # Calcular tamaño de salida del encoder después del pooling
        with torch.no_grad():
            dummy_input = torch.zeros(1, n_channels, seq_len)
            encoder_output = self.encoder(dummy_input)
            # encoder_output shape: (batch, out_channels, seq_len_reduced)
            self.encoded_seq_len = encoder_output.shape[2]
            self.encoded_channels = encoder_output.shape[1]
        
        # Asegurar que el último canal del encoder coincide con latent_channels
        if self.encoded_channels != latent_channels:
            # Agregar una capa de proyección
            self.latent_projection = nn.Conv1d(
                in_channels=self.encoded_channels,
                out_channels=latent_channels,
                kernel_size=1,
            )
            self.encoded_channels = latent_channels
        else:
            self.latent_projection = nn.Identity()
        
        # ========================================
        # Decoder: CNN1D
        # ========================================
        decoder_layers = []
        decoder_in_channels = latent_channels
        
        for i in range(self.num_dec_layers):
            # Upsampling (si está definido)
            if dec_upsample_sizes[i] is not None and dec_upsample_sizes[i] > 1:
                # Opción 1: ConvTranspose1d (más común)
                decoder_layers.append(
                    nn.ConvTranspose1d(
                        in_channels=decoder_in_channels,
                        out_channels=decoder_in_channels,
                        kernel_size=dec_upsample_sizes[i],
                        stride=dec_upsample_sizes[i],
                        padding=0,
                        output_padding=0,
                    )
                )
            
            # Conv1d
            decoder_layers.append(
                nn.Conv1d(
                    in_channels=decoder_in_channels,
                    out_channels=dec_out_channels[i],
                    kernel_size=dec_kernel_sizes[i],
                    padding=(dec_kernel_sizes[i] - 1) // 2,  # padding 'same'
                )
            )
            
            # BatchNorm (si está habilitado)
            if use_batchnorm and i < self.num_dec_layers - 1:  # No BatchNorm en la última capa
                decoder_layers.append(nn.BatchNorm1d(dec_out_channels[i]))
            
            # Activación (tanh en la última capa para normalizar salida)
            if i == self.num_dec_layers - 1:
                decoder_layers.append(nn.Tanh())
            else:
                if cnn_activation.lower() == "relu":
                    decoder_layers.append(nn.ReLU())
                elif cnn_activation.lower() == "leakyrelu":
                    decoder_layers.append(nn.LeakyReLU(0.1))
            
            decoder_in_channels = dec_out_channels[i]
        
        self.decoder = nn.Sequential(*decoder_layers)
        
        # Ajustar tamaño final si es necesario
        with torch.no_grad():
            dummy_latent = torch.zeros(1, latent_channels, self.encoded_seq_len)
            decoder_output = self.decoder(dummy_latent)
            if decoder_output.shape[2] != seq_len:
                # Agregar capa de ajuste final
                final_scale = seq_len / decoder_output.shape[2]
                if abs(final_scale - 1.0) > 0.01:
                    self.final_adjust = nn.Upsample(size=seq_len, mode='linear', align_corners=False)
                else:
                    self.final_adjust = nn.Identity()
            else:
                self.final_adjust = nn.Identity()
        
    def encode(self, x: torch.Tensor) -> torch.Tensor:
        """
        Encoder: x -> representación latente
        
        Args:
            x: Tensor de forma (batch_size, seq_len, n_channels) o (batch_size, n_channels, seq_len)
        
        Returns:
            Tensor de forma (batch_size, latent_channels, encoded_seq_len)
        """
        # Asegurar que x tiene forma (batch_size, n_channels, seq_len) para Conv1d
        if len(x.shape) == 3 and x.shape[1] == self.seq_len and x.shape[2] == self.n_channels:
            # Forma (batch, seq_len, channels) -> transponer a (batch, channels, seq_len)
            x = x.transpose(1, 2)
        
        # Encoder CNN1D
        encoded = self.encoder(x)  # (batch, encoded_channels, encoded_seq_len)
        encoded = self.latent_projection(encoded)  # (batch, latent_channels, encoded_seq_len)
        
        return encoded
    
    def decode(self, latent: torch.Tensor) -> torch.Tensor:
        """
        Decoder: representación latente -> reconstrucción
        
        Args:
            latent: Tensor de forma (batch_size, latent_channels, encoded_seq_len)
        
        Returns:
            Tensor de forma (batch_size, n_channels, seq_len)
        """
        # Decoder CNN1D
        decoded = self.decoder(latent)  # (batch, n_channels, seq_len_approx)
        decoded = self.final_adjust(decoded)  # Ajustar a seq_len exacto
        
        return decoded
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass completo: encode -> decode
        
        Args:
            x: Tensor de forma (batch_size, seq_len, n_channels) o (batch_size, n_channels, seq_len)
        
        Returns:
            Tensor reconstruido de forma (batch_size, seq_len, n_channels)
        """
        # Asegurar formato (batch, channels, seq_len) para el encoder
        original_shape = x.shape
        if len(original_shape) == 3 and original_shape[1] == self.seq_len and original_shape[2] == self.n_channels:
            x = x.transpose(1, 2)  # (batch, channels, seq_len)
        
        latent = self.encode(x)
        reconstructed = self.decode(latent)  # (batch, channels, seq_len)
        
        # Transponer de vuelta a (batch, seq_len, channels)
        reconstructed = reconstructed.transpose(1, 2)
        
        return reconstructed
    
    def get_reconstruction_error(self, x: torch.Tensor) -> torch.Tensor:
        """
        Calcula el error de reconstrucción (MSE) para cada muestra.
        
        Args:
            x: Tensor de forma (batch_size, seq_len, n_channels)
        
        Returns:
            Tensor de forma (batch_size,) con el error de reconstrucción por muestra
        """
        reconstructed = self.forward(x)
        # Calcular MSE por muestra
        mse = torch.mean((x - reconstructed) ** 2, dim=(1, 2))  # (batch_size,)
        return mse


In [8]:
# ========================================
# Instanciar modelo
# ========================================
def create_model(config: Dict) -> CNN1D_Autoencoder:
    """Crea e instancia el modelo Autoencoder CNN1D."""
    model = CNN1D_Autoencoder(
        n_channels=config["N_CHANNELS"],
        seq_len=config["SEQ_LEN"],
        enc_out_channels=config["enc_out_channels"],
        enc_kernel_sizes=config["enc_kernel_sizes"],
        enc_pool_sizes=config["enc_pool_sizes"],
        dec_out_channels=config["dec_out_channels"],
        dec_kernel_sizes=config["dec_kernel_sizes"],
        dec_upsample_sizes=config["dec_upsample_sizes"],
        use_batchnorm=config["USE_BATCHNORM"],
        cnn_activation=config["CNN_ACTIVATION"],
        latent_channels=config["LATENT_CHANNELS"],
    )
    
    # Contar parámetros
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"✓ Modelo creado:")
    print(f"  Parámetros totales: {total_params:,} ({total_params / 1e6:.2f}M)")
    print(f"  Parámetros entrenables: {trainable_params:,}")
    print(f"  Encoded shape: (batch, {model.encoded_channels}, {model.encoded_seq_len})")
    print(f"  Latent channels: {config['LATENT_CHANNELS']}")
    
    return model


---

## 5. 🏋️ Funciones de Entrenamiento y Evaluación


---

## 6. 📊 Integración con MLflow


In [9]:
# ========================================
# Función de entrenamiento por época
# ========================================
def train_one_epoch(
    model: CNN1D_Autoencoder,
    train_loader: DataLoader,
    optimizer: optim.Optimizer,
    criterion: nn.Module,
    device: torch.device,
) -> float:
    """
    Entrena el modelo por una época (reconstrucción).
    
    Returns:
        loss_promedio
    """
    model.train()
    total_loss = 0.0
    total = 0
    
    for batch_x, _ in train_loader:  # Ignorar labels en entrenamiento no supervisado
        batch_x = batch_x.to(device, non_blocking=True)
        
        # Forward pass: reconstrucción
        reconstructed = model(batch_x)
        
        # Calcular pérdida (MSE entre original y reconstruido)
        loss = criterion(reconstructed, batch_x)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Acumular métricas
        total_loss += loss.item() * batch_x.size(0)
        total += batch_x.size(0)
    
    avg_loss = total_loss / total if total > 0 else 0.0
    return avg_loss


# ========================================
# Función de evaluación (reconstrucción)
# ========================================
def evaluate_reconstruction(
    model: CNN1D_Autoencoder,
    dataloader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
) -> Tuple[float, np.ndarray]:
    """
    Evalúa el modelo en un dataloader (reconstrucción).
    
    Returns:
        Tupla con (loss_promedio, errores_reconstrucción)
    """
    model.eval()
    total_loss = 0.0
    total = 0
    
    all_errors = []
    
    with torch.no_grad():
        for batch_x, _ in dataloader:
            batch_x = batch_x.to(device, non_blocking=True)
            
            # Forward pass: reconstrucción
            reconstructed = model(batch_x)
            loss = criterion(reconstructed, batch_x)
            
            # Calcular error de reconstrucción por muestra
            errors = model.get_reconstruction_error(batch_x)
            all_errors.append(errors.cpu().numpy())
            
            # Acumular métricas
            total_loss += loss.item() * batch_x.size(0)
            total += batch_x.size(0)
    
    avg_loss = total_loss / total if total > 0 else 0.0
    all_errors = np.concatenate(all_errors)
    
    return avg_loss, all_errors


# ========================================
# Función para calcular métricas de clasificación desde errores de reconstrucción
# ========================================
def compute_metrics_from_errors(
    reconstruction_errors: np.ndarray,
    y_true: np.ndarray,
    threshold: float,
) -> Dict[str, float]:
    """
    Calcula métricas de clasificación usando errores de reconstrucción y un umbral.
    
    Args:
        reconstruction_errors: Array con errores de reconstrucción (1D)
        y_true: Array con etiquetas verdaderas (0=normal, 1=anómalo)
        threshold: Umbral para clasificar anomalías
    
    Returns:
        Diccionario con métricas
    """
    # Predicciones: error > threshold => anómalo (1)
    y_pred = (reconstruction_errors > threshold).astype(int)
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # Matriz de confusión
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    if cm.size == 4:
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / max(1, tn + fp)  # TNR
        sensitivity = tp / max(1, tp + fn)   # TPR
    else:
        specificity = 0.0
        sensitivity = 0.0
    
    # ROC AUC (opcional, requiere probabilidades)
    try:
        # Usar errores normalizados como "probabilidades"
        errors_normalized = (reconstruction_errors - reconstruction_errors.min()) / (reconstruction_errors.max() - reconstruction_errors.min() + 1e-8)
        roc_auc = roc_auc_score(y_true, errors_normalized)
    except:
        roc_auc = 0.0
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "specificity": specificity,
        "sensitivity": sensitivity,
        "roc_auc": roc_auc,
        "confusion_matrix": cm,
    }


In [10]:
# ========================================
# Configuración de MLflow
# ========================================
def setup_mlflow(config: Dict) -> str:
    """
    Configura MLflow y crea/obtiene el experimento.
    
    Returns:
        ID del experimento
    """
    # Configurar tracking URI
    if config.get("MLFLOW_TRACKING_URI") is not None:
        mlflow.set_tracking_uri(config["MLFLOW_TRACKING_URI"])
    else:
        # Usar sqlite en el directorio padre
        PARENT_DIR = Path.cwd().parent.resolve()
        TRACKING_DB = (PARENT_DIR / "mlflow.db").resolve()
        mlflow.set_tracking_uri(f"sqlite:///{TRACKING_DB.as_posix()}")
        print(f"✓ MLflow tracking URI: sqlite:///{TRACKING_DB.as_posix()}")
    
    # Crear o obtener experimento
    experiment_name = config["EXPERIMENT_NAME"]
    
    try:
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Crear directorio de artefactos
            PARENT_DIR = Path.cwd().parent.resolve()
            ARTIFACT_ROOT = (PARENT_DIR / "mlflow_artifacts").resolve()
            ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)
            experiment_id = mlflow.create_experiment(experiment_name, artifact_location=ARTIFACT_ROOT.as_uri())
            print(f"✓ Experimento MLflow creado: {experiment_name} (ID: {experiment_id})")
            print(f"  Artifact root: {ARTIFACT_ROOT.as_uri()}")
        else:
            experiment_id = experiment.experiment_id
            print(f"✓ Experimento MLflow existente: {experiment_name} (ID: {experiment_id})")
    except Exception as e:
        print(f"⚠ Error al configurar MLflow: {e}")
        experiment_id = mlflow.set_experiment(experiment_name)
    
    return experiment_id


# ========================================
# Función para guardar matriz de confusión
# ========================================
def save_confusion_matrix(
    cm: np.ndarray,
    output_dir: Path,
    tag: str,
) -> Tuple[Path, Path]:
    """
    Guarda la matriz de confusión como PNG y CSV.
    
    Returns:
        Tupla con rutas (png_path, csv_path)
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Guardar como CSV
    csv_path = output_dir / f"confusion_matrix_{tag}.csv"
    df_cm = pd.DataFrame(cm, index=["Normal", "Anómalo"], columns=["Normal", "Anómalo"])
    df_cm.to_csv(csv_path)
    
    # Guardar como PNG
    png_path = output_dir / f"confusion_matrix_{tag}.png"
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=["Normal", "Anómalo"],
                yticklabels=["Normal", "Anómalo"])
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Real")
    ax.set_title(f"Matriz de Confusión - {tag}")
    plt.tight_layout()
    plt.savefig(png_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    return png_path, csv_path


# ========================================
# Función para guardar curvas de entrenamiento
# ========================================
def save_training_curves(
    train_losses: List[float],
    val_losses: List[float],
    output_dir: Path,
) -> Path:
    """Guarda las curvas de entrenamiento."""
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    epochs = range(1, len(train_losses) + 1)
    ax.plot(epochs, train_losses, 'b-', label='Train Loss', linewidth=2)
    ax.plot(epochs, val_losses, 'r-', label='Val Loss', linewidth=2)
    ax.set_xlabel('Época')
    ax.set_ylabel('Loss (MSE)')
    ax.set_title('Curvas de Entrenamiento - Autoencoder CNN1D')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    
    curves_path = output_dir / "training_curves.png"
    plt.savefig(curves_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    return curves_path


In [11]:
# ========================================
# Función para seleccionar umbral de anomalía
# ========================================
def select_threshold(
    reconstruction_errors: np.ndarray,
    method: str = "percentile",
    percentile: float = 95.0,
) -> float:
    """
    Selecciona un umbral de anomalía basado en errores de reconstrucción.
    
    Args:
        reconstruction_errors: Array con errores de reconstrucción
        method: Método para seleccionar umbral ("percentile" o "mean_std")
        percentile: Percentil a usar si method="percentile"
    
    Returns:
        Umbral seleccionado
    """
    if method == "percentile":
        threshold = np.percentile(reconstruction_errors, percentile)
        print(f"✓ Umbral seleccionado (percentil {percentile}): {threshold:.6f}")
    elif method == "mean_std":
        mean = np.mean(reconstruction_errors)
        std = np.std(reconstruction_errors)
        threshold = mean + 2 * std  # 2 desviaciones estándar
        print(f"✓ Umbral seleccionado (mean + 2*std): {threshold:.6f}")
    else:
        raise ValueError(f"Método {method} no soportado")
    
    return threshold


# ========================================
# Tarea Prefect: Cargar datos
# ========================================
@task(name="load_data", log_prints=True, cache_policy=NO_CACHE)
def task_load_data(config: Dict) -> Tuple[DataLoader, DataLoader, DataLoader, torch.Tensor, torch.Tensor]:
    """Tarea Prefect para cargar datos."""
    print("📂 Cargando datos...")
    X_train_normals, y_train_normals, X_val, y_val, X_test, y_test = load_tensor_data(config["DATA_DIR"])
    
    train_loader, val_loader, test_loader = create_dataloaders_from_tensors(
        X_train_normals, y_train_normals, X_val, y_val, X_test, y_test,
        batch_size=config["BATCH_SIZE"],
        shuffle_train=True,
    )
    
    return train_loader, val_loader, test_loader, y_val, y_test


# ========================================
# Tarea Prefect: Entrenar modelo
# ========================================
@task(name="train_model", log_prints=True, cache_policy=NO_CACHE)
def task_train_model(
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    config: Dict,
    experiment_id: str,
) -> Tuple[CNN1D_Autoencoder, List[float], List[float], np.ndarray]:
    """Tarea Prefect para entrenar el modelo."""
    print("🧠 Creando modelo...")
    model = create_model(config)
    model = model.to(device)
    print(f"  ✓ Modelo en {device}")
    
    # Optimizador y criterio
    print(f"  🔄 Inicializando optimizador y criterio...")
    optimizer = optim.Adam(
        model.parameters(),
        lr=config["LEARNING_RATE"],
        weight_decay=config["WEIGHT_DECAY"],
    )
    criterion = nn.MSELoss()
    print(f"  ✓ Optimizador y criterio listos")
    
    # Listas para tracking
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_state = None
    
    # Iniciar run de MLflow
    print(f"  🔄 Iniciando run de MLflow...")
    with mlflow.start_run(experiment_id=experiment_id, run_name=config["RUN_NAME"]):
        print(f"  ✓ Run de MLflow iniciado")
        
        # Log hiperparámetros
        print(f"  🔄 Loggeando hiperparámetros en MLflow...")
        mlflow.log_params({
            "n_channels": config["N_CHANNELS"],
            "seq_len": config["SEQ_LEN"],
            "enc_out_channels": str(config["enc_out_channels"]),
            "enc_kernel_sizes": str(config["enc_kernel_sizes"]),
            "enc_pool_sizes": str(config["enc_pool_sizes"]),
            "dec_out_channels": str(config["dec_out_channels"]),
            "dec_kernel_sizes": str(config["dec_kernel_sizes"]),
            "dec_upsample_sizes": str(config["dec_upsample_sizes"]),
            "use_batchnorm": config["USE_BATCHNORM"],
            "cnn_activation": config["CNN_ACTIVATION"],
            "latent_channels": config["LATENT_CHANNELS"],
            "batch_size": config["BATCH_SIZE"],
            "learning_rate": config["LEARNING_RATE"],
            "num_epochs": config["NUM_EPOCHS"],
            "weight_decay": config["WEIGHT_DECAY"],
            "threshold_method": config["THRESHOLD_METHOD"],
            "threshold_percentile": config["THRESHOLD_PERCENTILE"],
            "seed": config["SEED"],
        })
        print(f"  ✓ Hiperparámetros loggeados")
        
        # Loop de entrenamiento
        print(f"\n🚀 Iniciando loop de entrenamiento ({config['NUM_EPOCHS']} épocas)...\n")
        for epoch in range(1, config["NUM_EPOCHS"] + 1):
            print(f"\n{'='*60}")
            print(f"📅 ÉPOCA {epoch}/{config['NUM_EPOCHS']}")
            print(f"{'='*60}")
            
            # Entrenar
            print(f"  🏋️ Entrenando...")
            train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
            train_losses.append(train_loss)
            print(f"  ✓ Entrenamiento completado: Loss={train_loss:.6f}")
            
            # Validar
            print(f"  📊 Validando...")
            val_loss, val_errors = evaluate_reconstruction(model, val_loader, criterion, device)
            val_losses.append(val_loss)
            print(f"  ✓ Validación completada: Loss={val_loss:.6f}")
            
            # Log métricas en MLflow (incluyendo error de reconstrucción promedio)
            avg_reconstruction_error = np.mean(val_errors)
            mlflow.log_metrics({
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_reconstruction_error": avg_reconstruction_error,
            }, step=epoch)
            
            # Guardar mejor modelo
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict().copy()
                print(f"  ⭐ Nuevo mejor modelo (val_loss={val_loss:.6f})")
            
            # Print progreso
            if epoch % 5 == 0 or epoch == 1:
                print(
                    f"Epoch {epoch:03d}/{config['NUM_EPOCHS']} | "
                    f"Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | Val Recon Error: {avg_reconstruction_error:.6f}"
                )
        
        # Cargar mejor modelo
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
            print(f"\n✓ Mejor modelo cargado (val_loss={best_val_loss:.6f})")
        
        # Guardar curvas de entrenamiento
        curves_path = save_training_curves(train_losses, val_losses, config["OUTPUT_DIR"])
        mlflow.log_artifact(str(curves_path))
        
        # Guardar modelo
        mlflow.pytorch.log_model(model, "model")
        
        print(f"✓ Entrenamiento completado. Mejor val_loss: {best_val_loss:.6f}")
    
    return model, train_losses, val_losses, val_errors


# ========================================
# Tarea Prefect: Seleccionar umbral
# ========================================
@task(name="select_threshold", log_prints=True, cache_policy=NO_CACHE)
def task_select_threshold(
    val_errors: np.ndarray,
    config: Dict,
    experiment_id: str,
) -> float:
    """Tarea Prefect para seleccionar umbral."""
    print("🔍 Seleccionando umbral de anomalía...")
    threshold = select_threshold(
        val_errors,
        method=config["THRESHOLD_METHOD"],
        percentile=config["THRESHOLD_PERCENTILE"],
    )
    
    # Log en MLflow
    with mlflow.start_run(experiment_id=experiment_id, run_name=config["RUN_NAME"]):
        mlflow.log_param("selected_threshold", threshold)
        mlflow.log_metric("threshold_value", threshold)
    
    return threshold


# ========================================
# Tarea Prefect: Evaluar en validación y test
# ========================================
@task(name="evaluate_final", log_prints=True, cache_policy=NO_CACHE)
def task_evaluate_final(
    model: CNN1D_Autoencoder,
    val_loader: DataLoader,
    test_loader: DataLoader,
    y_val: torch.Tensor,
    y_test: torch.Tensor,
    threshold: float,
    device: torch.device,
    config: Dict,
    experiment_id: str,
) -> Dict:
    """Tarea Prefect para evaluar en validación y test."""
    print("📊 Evaluando en validación y test...")
    
    model = model.to(device)
    criterion = nn.MSELoss()
    
    # Evaluar en validación
    print("  📊 Evaluando en validación...")
    val_loss, val_errors = evaluate_reconstruction(model, val_loader, criterion, device)
    val_metrics = compute_metrics_from_errors(val_errors, y_val.numpy(), threshold)
    
    # Evaluar en test
    print("  📊 Evaluando en test...")
    test_loss, test_errors = evaluate_reconstruction(model, test_loader, criterion, device)
    test_metrics = compute_metrics_from_errors(test_errors, y_test.numpy(), threshold)
    
    # Guardar matrices de confusión
    cm_val_path, _ = save_confusion_matrix(val_metrics["confusion_matrix"], config["OUTPUT_DIR"], "val")
    cm_test_path, _ = save_confusion_matrix(test_metrics["confusion_matrix"], config["OUTPUT_DIR"], "test")
    
    # Log en MLflow
    with mlflow.start_run(experiment_id=experiment_id, run_name=config["RUN_NAME"]):
        # Métricas de validación
        mlflow.log_metrics({
            "val_final_loss": val_loss,
            "val_final_reconstruction_error": np.mean(val_errors),
            "val_accuracy": val_metrics["accuracy"],
            "val_precision": val_metrics["precision"],
            "val_recall": val_metrics["recall"],
            "val_f1": val_metrics["f1"],
            "val_specificity": val_metrics["specificity"],
            "val_sensitivity": val_metrics["sensitivity"],
            "val_roc_auc": val_metrics["roc_auc"],
        })
        
        # Métricas de test
        mlflow.log_metrics({
            "test_final_loss": test_loss,
            "test_final_reconstruction_error": np.mean(test_errors),
            "test_accuracy": test_metrics["accuracy"],
            "test_precision": test_metrics["precision"],
            "test_recall": test_metrics["recall"],
            "test_f1": test_metrics["f1"],
            "test_specificity": test_metrics["specificity"],
            "test_sensitivity": test_metrics["sensitivity"],
            "test_roc_auc": test_metrics["roc_auc"],
        })
        
        # Artefactos
        mlflow.log_artifact(str(cm_val_path))
        mlflow.log_artifact(str(cm_test_path))
    
    print(f"\n✓ Evaluación completada:")
    print(f"  Validación - Acc: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1']:.4f}")
    print(f"  Test - Acc: {test_metrics['accuracy']:.4f}, F1: {test_metrics['f1']:.4f}")
    
    return {
        "val_metrics": val_metrics,
        "test_metrics": test_metrics,
        "val_errors": val_errors,
        "test_errors": test_errors,
    }


In [12]:
# ========================================
# Flujo principal de Prefect
# ========================================
@flow(name="cnn1d_autoencoder_anomaly_detection_flow", log_prints=True)
def main_flow(config: Dict, device: torch.device):
    """
    Flujo principal de Prefect que orquesta todo el proceso:
    1. Cargar datos
    2. Entrenar modelo
    3. Seleccionar umbral
    4. Evaluar en validación y test
    """
    print("🚀 Iniciando flujo de entrenamiento Autoencoder CNN1D...")
    print(f"Experimento MLflow: {config['EXPERIMENT_NAME']}")
    
    # Configurar MLflow
    experiment_id = setup_mlflow(config)
    
    # 1. Cargar datos
    train_loader, val_loader, test_loader, y_val, y_test = task_load_data(config)
    
    # 2. Entrenar modelo
    model, train_losses, val_losses, val_errors = task_train_model(
        train_loader, val_loader, device, config, experiment_id
    )
    
    # 3. Seleccionar umbral
    threshold = task_select_threshold(val_errors, config, experiment_id)
    
    # 4. Evaluar en validación y test
    results = task_evaluate_final(
        model, val_loader, test_loader, y_val, y_test,
        threshold, device, config, experiment_id
    )
    
    # Guardar modelo y umbral
    model_path = config["OUTPUT_DIR"] / "autoencoder_model.pt"
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': config,
        'threshold': threshold,
    }, model_path)
    print(f"\n✓ Modelo guardado en: {model_path}")
    
    print(f"\n✅ Flujo completado exitosamente!")
    print(f"\nRevisa MLflow para ver todos los artefactos y métricas.")
    
    return model, threshold, results


# ========================================
# Ejecutar flujo completo
# ========================================
if __name__ == "__main__" or True:  # Ejecutar en notebook
    print("="*80)
    print("🚀 EJECUTANDO FLUJO COMPLETO DE ENTRENAMIENTO")
    print("="*80)
    
    # Ejecutar flujo
    model, threshold, results = main_flow(CONFIG, DEVICE)
    
    print("\n" + "="*80)
    print("✅ ENTRENAMIENTO Y EVALUACIÓN COMPLETADOS")
    print("="*80)
    print(f"\n📊 Resumen de resultados:")
    print(f"  Umbral seleccionado: {threshold:.6f}")
    print(f"  Validación - Acc: {results['val_metrics']['accuracy']:.4f}, F1: {results['val_metrics']['f1']:.4f}")
    print(f"  Test - Acc: {results['test_metrics']['accuracy']:.4f}, F1: {results['test_metrics']['f1']:.4f}")
    print(f"\n📁 Artefactos guardados en: {CONFIG['OUTPUT_DIR']}")
    print(f"📊 Revisa MLflow para más detalles: mlflow ui")


🚀 EJECUTANDO FLUJO COMPLETO DE ENTRENAMIENTO


INFO:prefect:Starting temporary server on http://127.0.0.1:8759
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.
INFO:prefect.flow_runs:Beginning flow run 'hopeful-lizard' for flow 'cnn1d_autoencoder_anomaly_detection_flow'
INFO:prefect.flow_runs:🚀 Iniciando flujo de entrenamiento Autoencoder CNN1D...
INFO:prefect.flow_runs:Experimento MLflow: ecg_cnn_autoencoder
INFO:prefect.flow_runs:✓ MLflow tracking URI: sqlite:///S:/Proyecto final/mlflow.db
2025/11/25 16:30:25 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/11/25 16:30:25 INFO mlflow.store.db.utils: Updating database tables
2025-11-25 16:30:25 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-11-25 16:30:25 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-11-25 16:30:26 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-11-25 16:30:26 INFO  [alembic.runtime.migration] Will assume n


✅ ENTRENAMIENTO Y EVALUACIÓN COMPLETADOS

📊 Resumen de resultados:
  Umbral seleccionado: 0.000541
  Validación - Acc: 0.6328, F1: 0.1265
  Test - Acc: 0.6334, F1: 0.1286

📁 Artefactos guardados en: outputs
📊 Revisa MLflow para más detalles: mlflow ui
